In [1]:
import pandas as pd
from collections import Counter
import seaborn as sns

df = pd.read_excel('/Users/tmxhs/0_LoL/sample.xlsx')

In [2]:
df.head(5)

,Unnamed: 0,game_type,result,time,champ,kill,death,assi,kda,level,cs,krate,line
0,0,솔랭,승리,27분 34초,르블랑,4,10,5,0.90:1,레벨15,144 (5.2),킬관여 25%,Mid
1,1,솔랭,승리,13분 40초,모르가나,3,0,5,Perfect,레벨10,98 (7.2),킬관여 57%,jug
2,2,솔랭,승리,33분 52초,모르가나,15,3,17,10.67:1,레벨18,248 (7.3),킬관여 80%,jug
3,3,솔랭,승리,24분 39초,리 신,5,2,13,9.00:1,레벨14,156 (6.3),킬관여 72%,jug
4,4,솔랭,승리,30분 12초,모르가나,3,4,14,4.25:1,레벨17,220 (7.3),킬관여 44%,jug


In [3]:
del df[df.columns[0]]

In [4]:
Counter(df['line'])

Counter({'Mid': 388,
         'jug': 363,
         'Top': 354,
         'Spt': 480,
         'Ad': 375,
         'No match': 40})

In [5]:
Counter(df['game_type'])

Counter({'솔랭': 1884, '무작위 총력전': 87, '일반': 14, '자유 5:5 랭크': 15})

In [6]:
Counter(df['result'])

Counter({'승리': 1129, '패배': 861, '다시하기': 10})

In [7]:
# 원딜, 솔랭, 다시하기 제외
df_Ad = df['line'] == 'Ad';
df_sol = df['game_type'] == '솔랭';
df_re_x = df['result'] != '다시하기';

In [8]:
df1 = df[df_Ad & df_sol & df_re_x]

In [9]:
df1.shape # 375 -> 343 으로 줄어듬

(343, 12)

In [10]:
# index reset
df1 = df1.reset_index(drop=True)

In [11]:
# 변수 유형 정리
level1 = df1.level.str.split('레벨').str[1]
cs1 = df1.cs.str.split('(').str[0]
krate1 = df1.krate.str.split('킬관여').str[1].str.split('%').str[0]
kda1 = df1.kda.str.split(':').str[0]

# kda Perfect 인경우 30로 지정(이유: 최대가 29)
for i in range(df1.shape[0]):
    if kda1[i] == 'Perfect':
        kda1[i] = '30.00'
        
df1['level1'] = level1
df1['cs1'] = cs1
df1['krate1'] = krate1
df1['kda1'] = kda1

df1['level1'] = df1['level1'].astype(int)
df1['cs1'] = df1['cs1'].astype(int)
df1['krate1'] = df1['krate1'].astype(int)
df1['kda1'] = df1['kda1'].astype(float)

#시간 변수
# 데이터프레임에서 특정 문자를 기준으로 문자 나누기
minute = df1.time.str.split('분').str[0]
second = df1.time.str.split('분').str[1].str.split('초').str[0]

# 데이터 프레임에 열 추가하기
df1['minute'] = minute
df1['second'] = second

# 데이터프레임에서 열 문자형에서 숫자형으로 변환
df1['minute'] = df1['minute'].astype(int)
df1['second'] = df1['second'].astype(int)

# 데이터 프레임 열끼리 계산하여 새로운 열 추가하기
df1['t_time'] = df1['minute'] * 60  + df1['second']

# 승 - 1, 패 - 0 으로 리코딩
result1 = []

for i in range(df1.shape[0]):
    if df1.result[i] == '승리':
        result1.append (1)
    else:
        result1.append (0)
        
df1['result1'] = result1

In [12]:
df1.shape

(343, 20)

In [13]:
df1.head()

,game_type,result,time,champ,kill,death,assi,kda,level,cs,krate,line,level1,cs1,krate1,kda1,minute,second,t_time,result1
0,솔랭,패배,25분 22초,카이사,5,5,9,2.80:1,레벨13,217 (8.6),킬관여 70%,Ad,13,217,70,2.80,25,22,1522,0
1,솔랭,승리,27분 43초,카이사,5,4,7,3.00:1,레벨15,209 (7.5),킬관여 41%,Ad,15,209,41,3.00,27,43,1663,1
2,솔랭,승리,28분 18초,바루스,16,3,10,8.67:1,레벨15,212 (7.5),킬관여 76%,Ad,15,212,76,8.67,28,18,1698,1
3,솔랭,승리,15분 57초,바루스,6,0,6,Perfect,레벨10,141 (8.8),킬관여 67%,Ad,10,141,67,30.00,15,57,957,1
4,솔랭,패배,25분 18초,카이사,6,8,5,1.38:1,레벨13,237 (9.4),킬관여 52%,Ad,13,237,52,1.38,25,18,1518,0


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   game_type  343 non-null    object 
 1   result     343 non-null    object 
 2   time       343 non-null    object 
 3   champ      343 non-null    object 
 4   kill       343 non-null    int64  
 5   death      343 non-null    int64  
 6   assi       343 non-null    int64  
 7   kda        343 non-null    object 
 8   level      343 non-null    object 
 9   cs         343 non-null    object 
 10  krate      343 non-null    object 
 11  line       343 non-null    object 
 12  level1     343 non-null    int32  
 13  cs1        343 non-null    int32  
 14  krate1     343 non-null    int32  
 15  kda1       343 non-null    float64
 16  minute     343 non-null    int32  
 17  second     343 non-null    int32  
 18  t_time     343 non-null    int32  
 19  result1    343 non-null    int64  
dtypes: float64

In [15]:
df2 = df1[['result1','champ','kill','death','assi','level1','cs1','krate1','kda1','t_time']]

In [16]:
# 열 이름 바꾸기 (Tip df2.columns 확인후 긁어오기)
df2.columns = ['result', 'champ', 'kill', 'death', 'assi', 'level', 'cs', 'krate',
       'kda', 'time']

In [17]:
df2

,result,champ,kill,death,assi,level,cs,krate,kda,time
0,0,카이사,5,5,9,13,217,70,2.80,1522
1,1,카이사,5,4,7,15,209,41,3.00,1663
2,1,바루스,16,3,10,15,212,76,8.67,1698
3,1,바루스,6,0,6,10,141,67,30.00,957
4,0,카이사,6,8,5,13,237,52,1.38,1518
...,...,...,...,...,...,...,...,...,...,...
338,0,카이사,3,6,2,11,171,42,0.83,1329
339,0,사미라,5,8,8,12,181,65,1.63,1305
340,1,바루스,6,2,5,10,134,46,5.50,918
341,0,바루스,4,7,5,10,139,64,1.29,1216


In [20]:
Counter(df2['champ']) # 11건

Counter({'카이사': 95,
         '바루스': 40,
         '코그모': 18,
         '징크스': 38,
         '칼리스타': 21,
         '드레이븐': 5,
         '진': 9,
         '자야': 3,
         '사미라': 32,
         '시비르': 1,
         '탐 켄치': 16,
         '카르마': 2,
         '트리스타나': 20,
         '이즈리얼': 16,
         '베인': 2,
         '야스오': 1,
         '세트': 4,
         '아펠리오스': 10,
         '이렐리아': 1,
         '갈리오': 1,
         '루시안': 3,
         '케이틀린': 1,
         '레넥톤': 1,
         '제라스': 1,
         '아트록스': 1,
         '킨드레드': 1})

In [21]:
# 원딜 챔피언이 아닌 경우 제외
ad_nochamp = ['야스오', '세트', '이렐리아', '갈리오', '레넥톤', '탐 켄치','제라스', '아트록스', '킨드레드', '카르마']
df3 = df2

for i in range(len(ad_nochamp)):

    df3 = df3[df3.champ != ad_nochamp[i]]
    
df3 = df3.reset_index(drop=True)

In [22]:
df3.shape # 29건 삭제 

(314, 10)

In [23]:
# 요약
df3.describe()

,result,kill,death,assi,level,cs,krate,kda,time
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,0.550955,6.235669,4.713376,6.194268,12.700637,195.745223,49.216561,5.054045,1425.031847
std,0.498191,4.227009,2.784143,3.697486,2.380606,59.144260,15.732719,6.885022,324.447634
min,0.000000,0.000000,0.000000,0.000000,4.000000,49.000000,10.000000,0.180000,616.000000
25%,0.000000,3.000000,3.000000,3.000000,11.000000,155.250000,40.000000,1.447500,1218.000000
50%,1.000000,6.000000,5.000000,6.000000,13.000000,193.000000,50.000000,3.000000,1401.500000
75%,1.000000,9.000000,6.000000,9.000000,14.000000,227.750000,59.750000,5.000000,1662.500000
max,1.000000,25.000000,17.000000,17.000000,18.000000,426.000000,100.000000,30.000000,2221.000000


In [24]:
corr = df3.corr(method = 'pearson') 
corr

,result,kill,death,assi,level,cs,krate,kda,time
result,1.000000,0.469145,-0.459329,0.401330,0.306532,0.194856,0.050355,0.434414,0.034205
kill,0.469145,1.000000,-0.090887,0.388928,0.524548,0.372875,0.514334,0.239264,0.321081
death,-0.459329,-0.090887,1.000000,0.129258,0.173077,0.060439,-0.050219,-0.624617,0.393020
assi,0.401330,0.388928,0.129258,1.000000,0.683553,0.494818,0.456334,0.028510,0.606923
level,0.306532,0.524548,0.173077,0.683553,1.000000,0.887155,0.317528,-0.033488,0.903487
cs,0.194856,0.372875,0.060439,0.494818,0.887155,1.000000,0.281007,0.010501,0.853386
krate,0.050355,0.514334,-0.050219,0.456334,0.317528,0.281007,1.000000,0.144251,0.241058
kda,0.434414,0.239264,-0.624617,0.028510,-0.033488,0.010501,0.144251,1.000000,-0.227129
time,0.034205,0.321081,0.393020,0.606923,0.903487,0.853386,0.241058,-0.227129,1.000000


In [25]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

corr_group = df3.groupby(df3['result']).corr(method = 'pearson') 
corr_group

kill     death      assi     level        cs     krate  \
result                                                                     
0      kill   1.000000  0.109278  0.486195  0.573299  0.448534  0.519662   
       death  0.109278  1.000000  0.249598  0.221556  0.048210 -0.173015   
       assi   0.486195  0.249598  1.000000  0.660054  0.510009  0.470250   
       level  0.573299  0.221556  0.660054  1.000000  0.914464  0.214820   
       cs     0.448534  0.048210  0.510009  0.914464  1.000000  0.169198   
       krate  0.519662 -0.173015  0.470250  0.214820  0.169198  1.000000   
       kda    0.149966 -0.331588  0.147975  0.041768  0.087920  0.456297   
       time   0.511331  0.261926  0.626439  0.940004  0.903365  0.112678   
1      kill   1.000000  0.190223  0.139288  0.402210  0.268530  0.623274   
       death  0.190223  1.000000  0.483322  0.496405  0.270841  0.111213   
       assi   0.139288  0.483322  1.000000  0.634042  0.432781  0.495292   
       level  0.402210  0.496405  0.634042  1.000000  0.863574  0.425987   
       cs     0.268530  0.270841  0.432781  0.863574  1.000000  0.385664   
       krate  0.623274  0.111213  0.495292  0.425987  0.385664  1.000000   
       kda    0.028048 -0.657877 -0.268269 -0.287397 -0.139307  0.064444   
       time   0.268478  0.614629  0.663700  0.938838  0.833749  0.364609   

                   kda      time  
result                            
0      kill   0.149966  0.511331  
       death -0.331588  0.261926  
       assi   0.147975  0.626439  
       level  0.041768  0.940004  
       cs     0.087920  0.903365  
       krate  0.456297  0.112678  
       kda    1.000000 -0.004991  
       time  -0.004991  1.000000  
1      kill   0.028048  0.268478  
       death -0.657877  0.614629  
       assi  -0.268269  0.663700  
       level -0.287397  0.938838  
       cs    -0.139307  0.833749  
       krate  0.064444  0.364609  
       kda    1.000000 -0.368216  
       time  -0.368216  1.000000

In [169]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [249]:
# 모든 변수(8)
lol_data = [[a, b, c, d, e, f, g, h] for a, b, c, d, e, f, g, h 
            in zip(df3['kill'], df3['death'], df3['assi'],df3['level'],df3['cs'],df3['krate'],df3['kda'],df3['time'])]

lol_target = [l for l in df3['result']]

In [250]:
input_arr = np.array(lol_data)
target_arr = np.array(lol_target)

train_input, test_input, train_target, test_target = train_test_split(input_arr, target_arr, stratify=target_arr, test_size=0.3 , random_state = 42)

In [251]:
lol_score_test = []
lol_score_train = []

for i in range(1, len(train_input)+1):
    kn = KNeighborsClassifier(n_neighbors = i)
    
    # train_set scaled    
    mean = np.mean(train_input, axis = 0) # 0 열 기준 
    std = np.std(train_input, axis = 0)
    train_scaled = (train_input - mean) / std
    
    kn.fit(train_scaled, train_target)

    # train_set scaled    
    test_scaled = (test_input - mean) / std
    
    lol_score_test.append(kn.score(test_scaled, test_target))
    lol_score_train.append(kn.score(train_scaled, train_target))
    
print('k =', lol_score_test.index(max(lol_score_test)) + 1, '일때 test점수가', round(max(lol_score_test),4), '로 가장높고',
     '이때, train점수가', round(lol_score_train[lol_score_test.index(max(lol_score_test))],4), '입니다.')

if lol_score_train[lol_score_test.index(max(lol_score_test))] > lol_score_train[lol_score_test.index(max(lol_score_test))]:
    print("이 모델은 과대적합입니다.")
else:
    print("따라서, 이 모델은 과소적합입니다.")

k = 20 일때 test점수가 0.9368 로 가장높고 이때, train점수가 0.895 입니다.
따라서, 이 모델은 과소적합입니다.


In [235]:
new = [[2, 5, 5, 9, 109, 100, 1.4, 920], 
            [5, 3, 4, 11, 159, 30, 3, 1122],
            [8, 2, 8, 12, 151, 62, 8, 1096],
            [4, 3, 3, 11, 170, 64, 2.33, 1248],
            [2, 5, 8, 10, 121, 59, 2, 1133]]

mean = np.mean(train_input, axis = 0) # 0 열 기준 
std = np.std(train_input, axis = 0)

new_scaled = ( (new - mean) / std)

In [236]:
kn = KNeighborsClassifier(n_neighbors = 20)
kn.fit(train_scaled, train_target)

kn.predict(new_scaled)

array([0, 1, 1, 0, 0])

In [237]:
# 변수(7)
import itertools

combi = itertools.combinations(range(0, 8), 7)
a = list((combi))

var = ['kill', 'death', 'assi', 'level', 'cs','krate' , 'kda', 'time']

for j in range(len(a)):
    lol_data = [[a, b, c, d, e, f, g] for a, b, c, d, e, f, g 
            in zip(df3[var[a[j][0]]], df3[var[a[j][1]]], df3[var[a[j][2]]],df3[var[a[j][3]]],
                   df3[var[a[j][4]]],df3[var[a[j][5]]],df3[var[a[j][6]]])]
    
    input_arr = np.array(lol_data)
    target_arr = np.array(lol_target)

    train_input, test_input, train_target, test_target = train_test_split(input_arr, target_arr, stratify=target_arr, test_size=0.3 , random_state = 42)
    
    lol_score_test = []
    lol_score_train = []

    for i in range(1, len(train_input)+1):
        kn = KNeighborsClassifier(n_neighbors = i)
    
        # train_set scaled    
        mean = np.mean(train_input, axis = 0) # 0 열 기준 
        std = np.std(train_input, axis = 0)
        train_scaled = (train_input - mean) / std
    
        kn.fit(train_scaled, train_target)

        # test_set scaled    
        test_scaled = (test_input - mean) / std
    
        lol_score_test.append(kn.score(test_scaled, test_target))
        lol_score_train.append(kn.score(train_scaled, train_target))
    
    print("사용한 변수는:", var[a[j][0]], var[a[j][1]], var[a[j][2]],var[a[j][3]],
                   var[a[j][4]], var[a[j][5]], var[a[j][6]], "입니다")
    print('k =', lol_score_test.index(max(lol_score_test)) + 1, '일때 test점수가', round(max(lol_score_test),4), '로 가장높고',
     '이때, train점수가', round(lol_score_train[lol_score_test.index(max(lol_score_test))],4), '입니다.')

    if lol_score_train[lol_score_test.index(max(lol_score_test))] > lol_score_test[lol_score_test.index(max(lol_score_test))]:
        print("따라서, 이 모델은 과대적합입니다.")
    else:
        print("따라서, 이 모델은 과소적합입니다.")
        
    print()

사용한 변수는: kill death assi level cs krate kda 입니다
k = 28 일때 test점수가 0.9263 로 가장높고 이때, train점수가 0.8813 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level cs krate time 입니다
k = 24 일때 test점수가 0.9263 로 가장높고 이때, train점수가 0.9087 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level cs kda time 입니다
k = 15 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.8813 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level krate kda time 입니다
k = 22 일때 test점수가 0.9263 로 가장높고 이때, train점수가 0.8858 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi cs krate kda time 입니다
k = 25 일때 test점수가 0.9158 로 가장높고 이때, train점수가 0.8767 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death level cs krate kda time 입니다
k = 37 일때 test점수가 0.8421 로 가장높고 이때, train점수가 0.8265 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill assi level cs krate kda time 입니다
k = 11 일때 test점수가 0.8526 로 가장높고 이때, train점수가 0.8904 입니다.
따라서, 이 모델은 과대적합입니다.

사용한 변수는: death assi level cs krate kda time 입니다
k = 28 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.8767 입니다.
따라서, 이 모델은 과소

In [266]:
# 사용한 변수는: kill assi level cs krate kda time 입니다
# k = 11 일때 test점수가 0.8526 로 가장높고 이때, train점수가 0.8904 입니다.
# 따라서, 이 모델은 과대적합입니다.
lol_data = [[a, b, c, d, e, f, g] for a, b, c, d, e, f, g
            in zip(df3['kill'], df3['assi'], df3['level'],df3['cs'],df3['krate'],df3['kda'], df3['time'])]

lol_target = [l for l in df3['result']]

In [267]:
input_arr = np.array(lol_data)
target_arr = np.array(lol_target)

train_input, test_input, train_target, test_target = train_test_split(input_arr, target_arr, stratify=target_arr, test_size=0.3 , random_state = 42)

# train_set scaled    
mean = np.mean(train_input, axis = 0) # 0 열 기준 
std = np.std(train_input, axis = 0)
train_scaled = (train_input - mean) / std

# test_set scaled    
test_scaled = (test_input - mean) / std

In [269]:
new = [[2, 5, 9, 109, 100, 1.4, 920], 
            [5, 4, 11, 159, 30, 3, 1122],
            [8, 8, 12, 151, 62, 8, 1096],
            [4, 3, 11, 170, 64, 2.33, 1248],
            [2, 8, 10, 121, 59, 2, 1133]]

mean = np.mean(train_input, axis = 0) # 0 열 기준 
std = np.std(train_input, axis = 0)

new_scaled = ( (new - mean) / std)

In [270]:
kn = KNeighborsClassifier(n_neighbors = 11)
kn.fit(train_scaled, train_target)

kn.predict(new_scaled)

array([0, 1, 1, 0, 0])

In [271]:
# 변수(7)
import itertools

combi = itertools.combinations(range(0, 8), 6)
a = list((combi))

var = ['kill', 'death', 'assi', 'level', 'cs','krate' , 'kda', 'time']

for j in range(len(a)):
    lol_data = [[a, b, c, d, e, f] for a, b, c, d, e, f 
            in zip(df3[var[a[j][0]]], df3[var[a[j][1]]], df3[var[a[j][2]]],df3[var[a[j][3]]],
                   df3[var[a[j][4]]],df3[var[a[j][5]]])]
    
    input_arr = np.array(lol_data)
    target_arr = np.array(lol_target)

    train_input, test_input, train_target, test_target = train_test_split(input_arr, target_arr, stratify=target_arr, test_size=0.3 , random_state = 42)
    
    lol_score_test = []
    lol_score_train = []

    for i in range(1, len(train_input)+1):
        kn = KNeighborsClassifier(n_neighbors = i)
    
        # train_set scaled    
        mean = np.mean(train_input, axis = 0) # 0 열 기준 
        std = np.std(train_input, axis = 0)
        train_scaled = (train_input - mean) / std
    
        kn.fit(train_scaled, train_target)

        # test_set scaled    
        test_scaled = (test_input - mean) / std
    
        lol_score_test.append(kn.score(test_scaled, test_target))
        lol_score_train.append(kn.score(train_scaled, train_target))
    
    print("사용한 변수는:", var[a[j][0]], var[a[j][1]], var[a[j][2]],var[a[j][3]],
                   var[a[j][4]], var[a[j][5]], "입니다")
    print('k =', lol_score_test.index(max(lol_score_test)) + 1, '일때 test점수가', round(max(lol_score_test),4), '로 가장높고',
     '이때, train점수가', round(lol_score_train[lol_score_test.index(max(lol_score_test))],4), '입니다.')

    if lol_score_train[lol_score_test.index(max(lol_score_test))] > lol_score_test[lol_score_test.index(max(lol_score_test))]:
        print("따라서, 이 모델은 과대적합입니다.")
    else:
        print("따라서, 이 모델은 과소적합입니다.")
        
    print()

사용한 변수는: kill death assi level cs krate 입니다
k = 49 일때 test점수가 0.8947 로 가장높고 이때, train점수가 0.8676 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level cs kda 입니다
k = 3 일때 test점수가 0.8737 로 가장높고 이때, train점수가 0.8995 입니다.
따라서, 이 모델은 과대적합입니다.

사용한 변수는: kill death assi level cs time 입니다
k = 15 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.8858 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level krate kda 입니다
k = 46 일때 test점수가 0.9158 로 가장높고 이때, train점수가 0.8767 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level krate time 입니다
k = 22 일때 test점수가 0.9263 로 가장높고 이때, train점수가 0.895 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi level kda time 입니다
k = 3 일때 test점수가 0.8947 로 가장높고 이때, train점수가 0.9269 입니다.
따라서, 이 모델은 과대적합입니다.

사용한 변수는: kill death assi cs krate kda 입니다
k = 32 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.8767 입니다.
따라서, 이 모델은 과소적합입니다.

사용한 변수는: kill death assi cs krate time 입니다
k = 22 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.9087 입니다.
따라서, 이 모델은 과대적합입니다.

사용한 변수는: kill death assi cs kd

In [273]:
# 사용한 변수는: kill death assi cs krate time 입니다
# k = 22 일때 test점수가 0.9053 로 가장높고 이때, train점수가 0.9087 입니다.
# 따라서, 이 모델은 과대적합입니다.
lol_data = [[a, b, c, d, e, f] for a, b, c, d, e, f
            in zip(df3['kill'], df3['death'], df3['assi'],df3['cs'],df3['krate'], df3['time'])]

lol_target = [l for l in df3['result']]

In [274]:
input_arr = np.array(lol_data)
target_arr = np.array(lol_target)

train_input, test_input, train_target, test_target = train_test_split(input_arr, target_arr, stratify=target_arr, test_size=0.3 , random_state = 42)

# train_set scaled    
mean = np.mean(train_input, axis = 0) # 0 열 기준 
std = np.std(train_input, axis = 0)
train_scaled = (train_input - mean) / std

# test_set scaled    
test_scaled = (test_input - mean) / std

In [275]:
new = [[2, 5, 5, 109, 100, 920], 
            [5, 3, 4, 159, 30, 1122],
            [8, 2, 8, 151, 62, 1096],
            [4, 3, 3, 170, 64, 1248],
            [2, 5, 8, 121, 59, 1133]]

mean = np.mean(train_input, axis = 0) # 0 열 기준 
std = np.std(train_input, axis = 0)

new_scaled = ( (new - mean) / std)

In [276]:
kn = KNeighborsClassifier(n_neighbors = 11)
kn.fit(train_scaled, train_target)

kn.predict(new_scaled)

array([0, 1, 1, 0, 0])

In [ ]:
############ 스케일전 

In [256]:
# 변수(6)

import itertools

combi = itertools.combinations(range(0, 8), 6)
a = list((combi))

var = ['kill', 'death', 'assi', 'level', 'cs','krate' , 'kda', 'time']

for j in range(len(a)):
    lol_data = [[a, b, c, d, e, f] for a, b, c, d, e, f 
            in zip(df3[var[a[j][0]]], df3[var[a[j][1]]], df3[var[a[j][2]]],df3[var[a[j][3]]],
                   df3[var[a[j][4]]],df3[var[a[j][5]]])]
    
    input_arr = np.array(lol_data)
    target_arr = np.array(lol_target)
    
    train_input, test_input, train_target, test_target = train_test_split(lol_data, lol_target, stratify=lol_target, random_state = 42)
    
    lol_score = []
    
    for i in range(1,236):
        kn = KNeighborsClassifier(n_neighbors = i)
        kn.fit(train_input, train_target)
        lol_score.append(kn.score(test_input, test_target))
    
    print("사용한 변수는:", var[a[j][0]], var[a[j][1]], var[a[j][2]],var[a[j][3]],
                   var[a[j][4]], var[a[j][5]], "입니다")
    print('k =', lol_score.index(max(lol_score)) + 1, '일때 점수가', max(lol_score), '로 가장높습니다')
    print()

사용한 변수는: kill death assi level cs krate 입니다
k = 24 일때 점수가 0.7341772151898734 로 가장높습니다

사용한 변수는: kill death assi level cs kda 입니다
k = 7 일때 점수가 0.8481012658227848 로 가장높습니다

사용한 변수는: kill death assi level cs time 입니다
k = 3 일때 점수가 0.6582278481012658 로 가장높습니다

사용한 변수는: kill death assi level krate kda 입니다
k = 7 일때 점수가 0.8734177215189873 로 가장높습니다

사용한 변수는: kill death assi level krate time 입니다
k = 46 일때 점수가 0.6329113924050633 로 가장높습니다

사용한 변수는: kill death assi level kda time 입니다
k = 46 일때 점수가 0.620253164556962 로 가장높습니다

사용한 변수는: kill death assi cs krate kda 입니다
k = 9 일때 점수가 0.7721518987341772 로 가장높습니다

사용한 변수는: kill death assi cs krate time 입니다
k = 3 일때 점수가 0.6708860759493671 로 가장높습니다

사용한 변수는: kill death assi cs kda time 입니다
k = 1 일때 점수가 0.6582278481012658 로 가장높습니다

사용한 변수는: kill death assi krate kda time 입니다
k = 49 일때 점수가 0.6329113924050633 로 가장높습니다

사용한 변수는: kill death level cs krate kda 입니다
k = 11 일때 점수가 0.759493670886076 로 가장높습니다

사용한 변수는: kill death level cs krate time 입니다
k = 55 일때 점수가 

In [257]:
# 사용한 변수는: kill death assi level krate kda 입니다
# k = 7 일때 점수가 0.8734177215189873 로 가장높습니다

lol_data = [[a, b, c, d, e, f] for a, b, c, d, e, f
            in zip(df3['kill'], df3['death'], df3['assi'],df3['level'],df3['krate'],df3['kda'])]

lol_target = [l for l in df3['result']]

In [258]:
input_arr = np.array(lol_data)
target_arr = np.array(lol_target)

train_input, test_input, train_target, test_target = train_test_split(lol_data, lol_target, stratify=lol_target, random_state = 42)

In [259]:
kn = KNeighborsClassifier(n_neighbors = 7)
kn.fit(train_input, train_target)

kn.predict([[2, 5, 5, 9, 100, 1.4], 
            [5, 3, 4, 11, 30, 3],
            [8, 2, 8, 12, 62, 8],
            [4, 3, 3, 11, 64, 2.33],
            [2, 5, 8, 10, 59, 2]],)

array([0, 1, 1, 0, 0])

In [260]:
kn.score(test_input, test_target)

0.8734177215189873